In [ ]:
import FaaSET
import FaaSRunner
import time
from datetime import datetime

In [ ]:

@FaaSET.cloud_function(platform="aws", config={"memory":128, "region": "us-west-1"})
def hello_world_faaset_128MB_c(request, context): 
    from SAAF import Inspector
    import time
    inspector = Inspector()
    inspector.inspectAll()
    time.sleep(request['time'])
    inspector.addAttribute("message", "Hello from the hotel lambda " + str(request["name"]) + "!")
    inspector.inspectAllDeltas()
    return inspector.finish() 

In [ ]:
cloud = hello_world_faaset_128MB_c({"name": "Steve", "time": 10}, None)
cloud

In [ ]:

@FaaSET.cloud_function(platform="aws", config={"memory":10000, "region": "us-west-1"})
def hello_world_faaset_10GB_c(request, context): 
    from SAAF import Inspector
    import time
    inspector = Inspector()
    inspector.inspectAll()
    time.sleep(request['time'])
    inspector.addAttribute("message", "Hello from the hotel lambda " + str(request["name"]) + "!")
    inspector.inspectAllDeltas()
    return inspector.finish() 

cloud = hello_world_faaset_10GB_c({"name": "Steve", "time": 10}, None)
cloud

# 24 hour test:

In [ ]:
start_time = time.time()
while time.time() - start_time < 90000:
    try:
        results_128 = FaaSRunner.experiment(
        
            function=hello_world_faaset_128MB_c, 
            runs_per_thread= 1,
            threads= 125,
            payloads=[{"name": "Bob", "time" : 10}],
            experiment_name="func_call_test_128MB_c")
        results_128['memory'] = '128MB'
        results_128['time_run'] = datetime.now().strftime("%m/%d/%Y %H:%M:%S")
        results_128.to_csv(f'results/results_24_hours_experiment_5_23_california_128MBvs10GB/result_128MB_call_{datetime.now().strftime("%H:%M:%S")}_california.csv', index=False )

        results_3GB = FaaSRunner.experiment(
        
            function=hello_world_faaset_10GB_c, 
            runs_per_thread= 1,
            threads= 125,
            payloads=[{"name": "Bob", "time" : 10}],
            experiment_name="func_call_test_10GB_c")
        results_3GB['memory'] = '10GB'
        results_3GB['time_run'] = datetime.now().strftime("%m/%d/%Y %H:%M:%S")
        results_3GB.to_csv(f'results/results_24_hours_experiment_5_23_california_128MBvs10GB/result_10GB_call_{datetime.now().strftime("%H:%M:%S")}_california.csv', index=False )
    except Exception:
        print("Exception occured: ", Exception)
    time.sleep(3600)

In [ ]:
results_128["cpuType"].value_counts()


In [ ]:
results_128["newcontainer"].value_counts()

In [ ]:
results_3008["cpuType"].value_counts()

In [ ]:
results_3008["newcontainer"].value_counts()